In [31]:
import os
import json
import geopandas as gpd
import numpy
import psycopg2
import random

import datetime

In [30]:
#SETTING UP preferencia DF

df = pd.read_excel('tmp_api/responses.xls')


list_caract = ['%_zona_verde', 'nivel_acustico', 'num_hospitales', 'num_colegios', 'num_chargestations','pm25','num_contenedores','num_transporte']

dict_caract = {3:'%_zona_verde' , 6:'nivel_acustico', 4:'num_hospitales', 2:'num_colegios', 8:'num_chargestations',5:'pm25',7:'num_contenedores',1:'num_transporte'}

#list_caract = [3, 6, 4, 2, 8, 5, 7, 1]

df = df.rename(columns={'¿Qué edad tienes?': 'Edad', '¿Tienes hijos?': 'Hijos', '¿Trabajas actualmente?': 'Trabajo', 
                        '¿Valoras en gran medida la existencia de comercios cerca de tu zona?': 'Comercios', 
                        '¿Valoras en gran medida la existencia de estaciones de transporte público cerca de tu zona?': list_caract[7], 
                        '¿Valoras en gran medida la existencia de lugares de ocio cerca de tu zona?': 'Ocio',
                        '¿Valoras en gran medida la existencia de colegios cerca de tu zona?': list_caract[3], 
                        '¿Valoras en gran medida la existencia de zonas verdes cerca de tu zona?': list_caract[0], 
                        '¿Valoras en gran medida la existencia de centros sanitarios cerca de tu zona?': list_caract[2], 
                        '¿Valoras negativamente la contaminación en tu zona?': list_caract[5], 
                        '¿El exceso de ruido supone un problema para ti?': list_caract[1], 
                        '¿Cuánto valoras la limpieza del barrio?': list_caract[6],
                        'Ante la posibilidad de adquirir un coche electrico, ¿valoras la existencia de puntos de recarga?': list_caract[4], 
                        'De las comodidades anteriores ¿cuáles serían las 3 que más valoras?': 'Comodidades', 
                        '¿Cuánto estarías dispuesto a pagar por el alquiler de una casa que ofrezca todas las comodidades que buscas?': 'Alquiler'})

df = df.drop(['Marca temporal', 'Puntuación','Comercios', 'Trabajo', 'Estado Civil', 'Ocio'], axis=1)

#adding a column for id
df.insert(0, 'id_cliente', range(1, len(df)+1))
#print(df.columns)
#print(df['Comodidades'])


caracter_df = pd.DataFrame()

caracter_df.insert(0, 'id_caract', range(1, len(dict_caract)+1))

caracter_df['nombre_caract'] = caracter_df['id_caract'].map(dict_caract)

# print(caracter_df)

cliente_df = pd.DataFrame()
cliente_df['id_cliente'] = df['id_cliente']

for i in list_caract:
    cliente_df[i] = df[i]



top3 = cliente_df.iloc[:, 1:].apply(lambda s: s.abs().nlargest(3).index.tolist(), axis=1) # selecciona el top 3 de preferencias



preferencia_df = pd.DataFrame()
preferencia_df['id_cliente'] = cliente_df['id_cliente']
preferencia_df = preferencia_df.assign(Recomendacion=top3)
preferencia_df[['Preferencia 1','Preferencia 2', 'Preferencia 3']] = pd.DataFrame(preferencia_df.Recomendacion.tolist(), index= preferencia_df.index)
preferencia_df = preferencia_df.drop(['Recomendacion'], axis = 1)
 

preferencia_df = pd.melt(preferencia_df, id_vars=['id_cliente'],
                         value_name="nombre_caract").drop(['variable'],axis=1).sort_values('id_cliente')

# preferencia_df = preferencia_df.rename(columns ={'Preferencia 1':'nombre'})

left_merged = pd.merge(
    preferencia_df, caracter_df, how="left", on=['nombre_caract']
)

preferencia_df['id_caract'] = left_merged['id_caract']

print(preferencia_df)

print(cliente_df)

     id_cliente       nombre_caract  id_caract
0             1      num_hospitales          4
320           1      nivel_acustico          6
160           1      num_transporte          3
1             2        %_zona_verde          6
321           2      num_hospitales          3
..          ...                 ...        ...
158         159      nivel_acustico          2
318         159                pm25          8
319         160  num_chargestations          5
159         160        %_zona_verde          2
479         160      nivel_acustico          6

[480 rows x 3 columns]
     id_cliente  %_zona_verde  nivel_acustico  num_hospitales  num_colegios  \
0             1             7             8.0               9             8   
1             2            10             6.0               9             1   
2             3             7             8.0               9             8   
3             4             7             5.0               5             1   
4             5  

In [32]:

# SETTING UP BARRIOS DF
path = '../datos/datos_out/barrios_caracteristicas_final.geojson'


#CARGAMOS LOS DATOS DE LOS BARRIOS
with open('../datos/datos_out/barrios_caracteristicas_final.geojson') as json_file:
    json_data = json.load(json_file)

barrios_json = []
for i in range(len(json_data['features'])):
    barrios_json.append(json_data['features'][i])           #Guardamos resto de campos de geojson


barrios_gpd = gpd.GeoDataFrame.from_features(barrios_json)
barrios_gpd.crs = 'epsg:4326' #Aseguramos que la proyección es la adecuada para coordenadas GPS

barrios_gpd['date_time'] = datetime.datetime.now().strftime("%Y/%m/%d %H:%M:%S")

barrios_gpd



,geometry,coddistrit,gis_gis_barrios_area,object_id_barrio,linkid,codbarrio,coddistbar,geo_point_2d,nombre_barrio,last_edited_user,...,id_caract_num_colegios,num_chargestations,id_caract_num_chargestations,pm25,id_caract_pm25,num_contenedores,id_caract_num_contenedores,num_transporte,id_caract_num_transporte,date_time
0,"POLYGON ((-0.33459 39.45478, -0.33326 39.45487...",11,917112.56250,62,0,5,115,"[39.44628964870906, -0.3326600366971329]",NATZARET,None,...,2,NaN,8,10.0,5,184.0,7,11.0,1,2022/11/25 11:37:56
1,"POLYGON ((-0.38124 39.45463, -0.38281 39.44951...",9,374887.53125,63,0,3,093,"[39.45082750748332, -0.3853982961775011]",LA CREU COBERTA,None,...,2,2.0,8,21.0,5,162.0,7,8.0,1,2022/11/25 11:37:56
2,"POLYGON ((-0.34709 39.47548, -0.34144 39.47379...",13,NaN,76,0,2,132,"[39.47198431553791, -0.3450427029615464]",CIUTAT JARDI,None,...,2,NaN,8,12.0,5,238.0,7,10.0,1,2022/11/25 11:37:56
3,"POLYGON ((-0.28767 39.55682, -0.28829 39.55654...",17,NaN,601,18,5,175,"[39.55880504953868, -0.3031699565111558]",RAFALELL-VISTABELLA,None,...,2,NaN,8,12.0,5,NaN,7,NaN,1,2022/11/25 11:37:56
4,"POLYGON ((-0.33151 39.48561, -0.33254 39.48046...",13,NaN,137,0,5,135,"[39.48035805242704, -0.3410905938386986]",LA CARRASCA,None,...,2,NaN,8,12.0,5,237.0,7,21.0,1,2022/11/25 11:37:56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,"POLYGON ((-0.39047 39.47825, -0.38996 39.47754...",3,496668.75000,28,0,3,033,"[39.473565513491664, -0.39098170775459257]",LA PETXINA,None,...,2,NaN,8,12.0,5,279.0,7,14.0,1,2022/11/25 11:37:56
84,"POLYGON ((-0.37567 39.47381, -0.37423 39.47351...",1,438658.31250,37,0,6,016,"[39.47010643870186, -0.3760489821791233]",SANT FRANCESC,None,...,2,2.0,8,12.0,5,580.0,7,27.0,1,2022/11/25 11:37:56
85,"POLYGON ((-0.35018 39.42223, -0.35011 39.42227...",19,NaN,249,0,3,193,"[39.406290326215334, -0.33967854582733203]",PINEDO,None,...,2,NaN,8,10.0,5,185.0,7,14.0,1,2022/11/25 11:37:56
86,"POLYGON ((-0.36720 39.50425, -0.36767 39.50338...",15,NaN,201,0,1,151,"[39.494312122911424, -0.3699310988383476]",ELS ORRIOLS,None,...,2,NaN,8,NaN,5,214.0,7,21.0,1,2022/11/25 11:37:56


In [27]:
#SETTING DATAFRAMES PER 

list_caract = ['%_zona_verde', 'nivel_acustico', 'num_hospitales', 'num_colegios', 'num_chargestations','pm25','num_contenedores','num_transporte']

barrios_top = pd.DataFrame(columns=['object_id_barrio', 'id_caract', 'val_carac'])

for i in range(len(list_caract)):
    data = barrios_gpd[['object_id_barrio', 'nombre_barrio','id_caract_'+list_caract[i], list_caract[i]]].sort_values([list_caract[i]], axis = 0, ascending = False).head(3).values.tolist()
    barrios_top = pd.concat([barrios_top,pd.DataFrame(data, columns=barrios_top.columns)],ignore_index=True)


print(barrios_top)

    object_id_barrio  id_caract   val_carac
0               17.0        3.0    0.451363
1               55.0        3.0    0.365194
2              185.0        3.0    0.319130
3               47.0        6.0    4.500000
4               12.0        6.0    4.500000
5               64.0        6.0    4.000000
6              169.0        4.0    3.000000
7                9.0        4.0    3.000000
8              585.0        4.0    3.000000
9               42.0        2.0   22.000000
10              49.0        2.0   22.000000
11             169.0        2.0   19.000000
12              31.0        8.0    6.000000
13              63.0        8.0    2.000000
14              24.0        8.0    2.000000
15             618.0        5.0   26.000000
16             169.0        5.0   26.000000
17               9.0        5.0   26.000000
18               9.0        7.0  831.000000
19              30.0        7.0  711.000000
20              37.0        7.0  580.000000
21              30.0        1.0 

In [28]:
left_merged = pd.merge(
    preferencia_df, barrios_top, how="left", on=['id_caract']
)
        
          
preferencia_df['object_id_barrio'] = left_merged['object_id_barrio']

print(preferencia_df)


     id_cliente       nombre_caract  id_caract  object_id_barrio
0             1      num_hospitales          4             169.0
320           1      nivel_acustico          6             185.0
160           1      num_transporte          3               9.0
1             2        %_zona_verde          6               9.0
321           2      num_hospitales          3              17.0
..          ...                 ...        ...               ...
158         159      nivel_acustico          2             169.0
318         159                pm25          8              17.0
319         160  num_chargestations          5              55.0
159         160        %_zona_verde          2             169.0
479         160      nivel_acustico          6             585.0

[480 rows x 4 columns]


In [29]:
# barrios_gpd = barrios_gpd[['object_id_barrio', '%_zona_verde', 'nivel_acustico', 'num_hospitales', 'num_colegios', 'num_chargestations', 'pm25', 'num_contenedores', 'num_transporte']]

# random_quality = ['Desfavorable', 'Regular', 'Razonablemente buena', 'Buena'] # Declaramos los niveles que obtenemos de los datos
# for i in barrios_gpd['pm25']:
    
#     barrios_gpd['pm25'] = random.choices(random_quality, k = len(barrios_gpd['pm25'])) # Generamos datos random para la calidad ambiental por cada barrio
        
# score = [1,2,3,4]
# punt_ambiental = []

# for i in barrios_gpd['pm25']:
    
#     if i == 'Desfavorable':
#         punt_ambiental.append(score[0])

#     elif i == 'Regular':
#         punt_ambiental.append(score[1])

#     elif i == 'Razonablemente buena':
#         punt_ambiental.append(score[2])

#     elif i == 'Buena':
#         punt_ambiental.append(score[3])

# barrios_gpd['punt_ambiental'] = punt_ambiental

# barrios_gpd = barrios_gpd.drop_duplicates(['object_id_barrio'])